In [1]:
import numpy as np
import helperfunctions as hf
import optimization as opt
import os
import matplotlib.pyplot as plt

# Beginning of Analysis on Data Collected from 3/1 - 3/9

In [2]:
dirs = (os.path.join('data', 'fitting_test_conditions_ramp'),
        os.path.join('data', 'fitting_noise_test_conditions_ramp'),
        os.path.join('data', 'fitting_test_conditions_triangle'),
        os.path.join('data', 'fitting_noise_test_conditions_triangle'))

In [41]:
project_dir = dirs[0]

cost_adjacency = np.zeros((4, 4))  # row index = number of real terms, col index = number of guess terms
early_cost_adjacency = np.zeros((4, 4))  # before half of the time
late_cost_adjacency = np.zeros((4, 4))  # after half of the time
elastic_term_error_adjacency = np.zeros((4, 4))  # stores the errors in the elastic stiffness terms
exact_tau_percent_error, exact_g_percent_error = np.zeros((4,)), np.zeros((4,))  # the errors in the terms when the number of terms == real num
compliant_cost_adjacency = np.zeros((4, 4))
stiff_cost_adjacency = np.zeros((4, 4))
mixed_stiffness_cost_adjacency = np.zeros((4, 4))

G1_cost_adjacency = np.zeros((4, 4))
compliant_G1_cost_adjacency = np.zeros((4, 4))
stiff_G1_cost_adjacency = np.zeros((4, 4))
mixed_G1_cost_adjacency = np.zeros((4, 4))

G2_cost_adjacency = np.zeros((4, 4))
compliant_G2_cost_adjacency = np.zeros((4, 4))
stiff_G2_cost_adjacency = np.zeros((4, 4))
mixed_G2_cost_adjacency = np.zeros((4, 4))

G1_early_cost_adjacency = np.zeros((4, 4))
G1_late_cost_adjacency = np.zeros((4, 4))

G2_early_cost_adjacency = np.zeros((4, 4))
G2_late_cost_adjacency = np.zeros((4, 4))

f = 2 * np.pi * np.logspace(0, 5, 1000)
f_early = 2 * np.pi * np.logspace(0, 2.5, 1000)
f_late = 2 * np.pi * np.logspace(2.5, 5, 1000)

for file in hf.get_files(project_dir):
    data = hf.load(file)
    Q_real, Q_final = data['Q_real'], data['Q_final']
    n_real, n_final = Q_real[1::2].size, Q_final[1::2].size
    cost_adjacency[n_real - 1, n_final - 1] += data['cost_final']
    
    h = data['indentation']
    t = data['time']
    t_mat_real, t_mat_final = opt.row2mat(t, Q_real[1::2].size), opt.row2mat(t, Q_final[1::2].size)
    R = data['tip_radius']
    # get force signal in time - get global error signal between force signal in ideal and fit
    f_real, f_final = opt.maxwell_force(Q_real, t_mat_real, t, h, R), opt.maxwell_force(Q_final, t_mat_final, t, h, R)
    # force error signal in time
    error_t = (f_real - f_final) ** 2
    
    early_cost_adjacency[n_real - 1, n_final - 1] += np.sum(error_t[:int(error_t.size / 2)])
    late_cost_adjacency[n_real - 1, n_final - 1] += np.sum(error_t[int(error_t.size / 2):])    

    elastic_term_error_adjacency[n_real - 1, n_final - 1] += abs(Q_real[0] - Q_final[0]) / Q_real[0]
    
    
    G1_cost, G2_cost = opt.maxwell_shear_sse(Q_real, Q_final, f)
    G1_cost_adjacency[n_real - 1, n_final - 1] += G1_cost
    G2_cost_adjacency[n_real - 1, n_final - 1] += G2_cost
    
    G1_cost_early, G2_cost_early = opt.maxwell_shear_sse(Q_real, Q_final, f_early)
    G1_cost_late, G2_cost_late = opt.maxwell_shear_sse(Q_real, Q_final, f_late)
    G1_early_cost_adjacency[n_real - 1, n_final - 1] += G1_cost_early
    G2_early_cost_adjacency[n_real - 1, n_final - 1] += G2_cost_early
    G1_late_cost_adjacency[n_real - 1, n_final - 1] += G1_cost_late
    G2_late_cost_adjacency[n_real - 1, n_final - 1] += G2_cost_late
    
    if n_real == n_final:
        exact_tau_percent_error[n_real - 1] += np.sum(abs(Q_real[2::2] - Q_final[2::2]) / Q_real[2::2])
        exact_g_percent_error[n_real - 1] += np.sum(abs(Q_real[1::2] - Q_final[1::2]) / Q_real[1::2])
    
    if Q_real[0] <= 50000.0 and all(Q_real[1::2] <= 50000.0):  # compliant
        compliant_cost_adjacency[n_real - 1, n_final - 1] += data['cost_final']
        compliant_G1_cost_adjacency[n_real - 1, n_final - 1] += G1_cost
        compliant_G2_cost_adjacency[n_real - 1, n_final - 1] += G2_cost
    
    if Q_real[0] >= 500000.0 and all(Q_real[1::2] >= 500000.0):  # stiff
        stiff_cost_adjacency[n_real - 1, n_final - 1] += data['cost_final']
        stiff_G1_cost_adjacency[n_real - 1, n_final - 1] += G1_cost
        stiff_G2_cost_adjacency[n_real - 1, n_final - 1] += G2_cost
    
    else:  # mixed
        mixed_stiffness_cost_adjacency[n_real - 1, n_final - 1] += data['cost_final']
        mixed_G1_cost_adjacency[n_real - 1, n_final - 1] += G1_cost
        mixed_G2_cost_adjacency[n_real - 1, n_final - 1] += G2_cost

print('early\n', compliant_cost_adjacency, '\nlate\n', stiff_cost_adjacency, np.sum(compliant_cost_adjacency), np.sum(stiff_cost_adjacency))

early
 [[1.04084125e-32 1.08103991e-28 4.06718855e-17 1.70819947e-08]
 [5.97554735e-28 1.03525644e-24 1.44604432e-24 1.68842283e-10]
 [2.49656855e-23 8.40117553e-25 6.17354893e-20 4.05356780e-10]
 [5.34141546e-16 6.02050173e-16 7.42799135e-16 4.29244002e-10]] 
late
 [[2.15355853e-36 5.74204862e-24 3.70998771e-22 1.55426788e-09]
 [1.49403623e-20 9.63784770e-21 6.53720573e-19 1.76466897e-10]
 [6.24204552e-16 9.69896711e-18 1.32187089e-16 3.59585665e-11]
 [5.37819165e-07 5.35894966e-07 5.11134057e-07 5.38848143e-07]] 1.8085439701412512e-08 2.1254630260367063e-06


In [ ]:
# plotting the derived error term (assuming the error in the time constant is small compared to the time constant itself)
from scipy.integrate import cumtrapz
h = data['indentation']
t = data['time']
R = data['tip_radius']
t_matrix = opt.row2mat(t, Q_real[1::2].size)
real_force = opt.maxwell_force(Q_real, t_matrix, t, h, R)
guess_force = opt.maxwell_force(Q_final, t_matrix, t, h, R)

error_terms = Q_final - Q_real
error_signal = cumtrapz(h**(3/2) * (error_terms[0] - np.sum(error_terms[1::2] / Q_final[2::2] * np.exp(- t_matrix / Q_final[2::2]))))

plt.plot(guess_force[1:] + error_signal)

# End of Analysis on Data Collected from 3/1 - 3/9

# Beginning of Simultaneous Fitting

# Beginning of Simultaneous Fitting

# Beginning of Training Number of Terms Model

# End of Training Number of Terms Model

In [ ]:
dirs = (os.path.join('data', 'fitting_no_noise_ramp'), os.path.join('data', 'fitting_noise_ramp'))

# No Noise Analysis

In [ ]:
t_error_global, t_error_low, t_error_high = [], [], []
f1_error_global, f1_error_low, f1_error_high = [], [], []
f2_error_global, f2_error_low, f2_error_high = [], [], []
for file in hf.get_files(dirs[0]):
    data = hf.load(file)
    Q_real, Q_final = data['Q_real'], data['Q_final']
    h = data['indentation']
    t = data['time']
    t_mat_real, t_mat_final = opt.row2mat(t, Q_real[1::2].size), opt.row2mat(t, Q_final[1::2].size)
    R = data['tip_radius']
    f = np.logspace(0, 4.5, t.size)
    f = np.linspace(1, 50e3, t.size)
    
    # get force signal in time - get global error signal between force signal in ideal and fit
    f_real, f_final = opt.maxwell_force(Q_real, t_mat_real, t, h, R), opt.maxwell_force(Q_final, t_mat_final, t, h, R)
    # force error signal in time
    error_t = (f_real - f_final)**2
    # full force error in time
    error_t_total = np.sum(error_t, axis=0)
    # force error in low time
    error_t_low = np.sum(error_t[: int(t.size / 2)], axis=0)
    # force error in high time
    error_t_high = np.sum(error_t[int(t.size / 2):], axis=0)
    
    # get moduli signal in frequency
    G1_real, G2_real = opt.maxwell_shear(Q_real, f)
    G1_final, G2_final = opt.maxwell_shear(Q_final, f)
    # full moduli error signal in frequency
    error_f1 = (G1_real - G1_final)**2
    error_f2 = (G2_real - G2_final)**2
    # full moduli error in frequency
    error_f1_total = np.sum(error_f1, axis=0)
    error_f2_total = np.sum(error_f2, axis=0)
    # moduli error in low frequency
    error_f1_low = np.sum(error_f1[: int(f.size / 2)], axis=0)
    error_f2_low = np.sum(error_f2[: int(f.size / 2)], axis=0)
    # moduli error in high frequency
    error_f1_high = np.sum(error_f1[int(f.size / 2):], axis=0)
    error_f2_high = np.sum(error_f2[int(f.size / 2):], axis=0)
    
    # store the errors in arrays
    t_error_global.append(error_t)
    t_error_low.append(error_t_low)
    t_error_high.append(error_t_high)
    f1_error_global.append(error_f1)
    f1_error_low.append(error_f1_low)
    f1_error_high.append(error_f1_high)
    f2_error_global.append(error_f2)
    f2_error_low.append(error_f2_low)
    f2_error_high.append(error_f2_high)

print('Force Error Low: {}\nForce Error High: {}\nLoss Modulus Low: {}\nLoss Modulus High: {}\nStorage Modulus Low: {}\nStorage Modulus High: {}'.format(
    sum(t_error_low), sum(t_error_high), sum(f1_error_low), sum(f1_error_high), sum(f2_error_low), sum(f2_error_high)
))

# relationship between time error and frequency error?
plt.plot(t, np.sum(t_error_global, axis=0))
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Error')
plt.title('Force Error in Time')
plt.show()

plt.plot(f, np.sum(f1_error_global, axis=0))
plt.grid()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Error')
plt.title('Loss Modulus Error in Frequency')
plt.xscale('log')
plt.show()

plt.plot(f, np.sum(f2_error_global, axis=0))
plt.grid()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Error')
plt.title('Storage Modulus Error in Frequency')
plt.xscale('log')
plt.show()

In [ ]:
t_error_global, t_error_low, t_error_high = [], [], []
f1_error_global, f1_error_low, f1_error_high = [], [], []
f2_error_global, f2_error_low, f2_error_high = [], [], []
for file in hf.get_files(dirs[1]):
    data = hf.load(file)
    Q_real, Q_final = data['Q_real'], data['Q_final']
    h = data['indentation']
    t = data['time']
    t_mat_real, t_mat_final = opt.row2mat(t, Q_real[1::2].size), opt.row2mat(t, Q_final[1::2].size)
    R = data['tip_radius']
    f = np.logspace(0, 4.5, t.size)
    f = np.linspace(1, 50e3, t.size)
    
    # get force signal in time - get global error signal between force signal in ideal and fit
    f_real, f_final = opt.maxwell_force(Q_real, t_mat_real, t, h, R), opt.maxwell_force(Q_final, t_mat_final, t, h, R)
    # force error signal in time
    error_t = (f_real - f_final)**2
    # full force error in time
    error_t_total = np.sum(error_t, axis=0)
    # force error in low time
    error_t_low = np.sum(error_t[: int(t.size / 2)], axis=0)
    # force error in high time
    error_t_high = np.sum(error_t[int(t.size / 2):], axis=0)
    
    # get moduli signal in frequency
    G1_real, G2_real = opt.maxwell_shear(Q_real, f)
    G1_final, G2_final = opt.maxwell_shear(Q_final, f)
    # full moduli error signal in frequency
    error_f1 = (G1_real - G1_final)**2
    error_f2 = (G2_real - G2_final)**2
    # full moduli error in frequency
    error_f1_total = np.sum(error_f1, axis=0)
    error_f2_total = np.sum(error_f2, axis=0)
    # moduli error in low frequency
    error_f1_low = np.sum(error_f1[: int(f.size / 2)], axis=0)
    error_f2_low = np.sum(error_f2[: int(f.size / 2)], axis=0)
    # moduli error in high frequency
    error_f1_high = np.sum(error_f1[int(f.size / 2):], axis=0)
    error_f2_high = np.sum(error_f2[int(f.size / 2):], axis=0)
    
    # store the errors in arrays
    t_error_global.append(error_t)
    t_error_low.append(error_t_low)
    t_error_high.append(error_t_high)
    f1_error_global.append(error_f1)
    f1_error_low.append(error_f1_low)
    f1_error_high.append(error_f1_high)
    f2_error_global.append(error_f2)
    f2_error_low.append(error_f2_low)
    f2_error_high.append(error_f2_high)

print('Force Error Low: {}\nForce Error High: {}\nLoss Modulus Low: {}\nLoss Modulus High: {}\nStorage Modulus Low: {}\nStorage Modulus High: {}'.format(
    sum(t_error_low), sum(t_error_high), sum(f1_error_low), sum(f1_error_high), sum(f2_error_low), sum(f2_error_high)
))

# relationship between time error and frequency error?
plt.plot(t, np.sum(t_error_global, axis=0))
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Error')
plt.title('Force Error in Time')
plt.show()

plt.plot(f, np.sum(f1_error_global, axis=0))
plt.grid()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Error')
plt.title('Loss Modulus Error in Frequency')
plt.xscale('log')
plt.show()

plt.plot(f, np.sum(f2_error_global, axis=0))
plt.grid()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Error')
plt.title('Storage Modulus Error in Frequency')
plt.xscale('log')
plt.show()